<a href="https://colab.research.google.com/github/AchmadZahrulAZ/C23-PS148-Capstone-Project/blob/machine-learning-development/29_May_Collaborative_Filtering_%5BFix(%3F)%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import Library

In [ ]:
!sudo pip3 install keras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import StringLookup
from tensorflow.keras.layers import Embedding, Flatten, Input, Concatenate, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import RootMeanSquaredError

#Preparing Dataset


#Cleaning Dataset

In [ ]:
mentee = pd.read_csv("Mentee.csv")
mentor = pd.read_csv("Mentors.csv")
rating = pd.read_csv("Ratings.csv")

In [ ]:
cols_mente = {"ID MENTEE" : "MenteeID", "SUBJECT" : "Subject" ,"NAME" : "MenteeName", "LOCATION" : "Location"}
mentee.rename(columns=cols_mente, inplace=True)
mentee_df = mentee.loc[0:1499, 'MenteeID' : 'Location']

In [ ]:
cols_mentor = {"ID MENTOR" :"TutorID",  "SUBJECT" : "Subject", "NAME" : "TutorName", "LOCATION" : "Location"}
mentor.rename(columns=cols_mentor, inplace=True)
mentor_df = mentor.loc[0:499, 'TutorID' : 'Location']

In [ ]:
cols_rating = {"ID MENTEE" : "MenteeID", "ID MENTOR" : "TutorID", "RATING" : "Rating"}
rating.rename(columns=cols_rating, inplace=True)
rating_df = rating.loc[0:12625, 'MenteeID' : 'Rating']
rating_df['Rating'] = rating_df['Rating'].astype(np.int64)

#Preprocessing Dataset

In [ ]:
tutor_rating = rating_df.merge(mentor_df, on="TutorID")
#tutor_rating = pd.merge(rating_df, mentor_df, on="TutorID")
tutor_rating

,MenteeID,TutorID,Rating,TutorName,Subject,Location
0,B36652,A16227,3,Andrianto,Ekonomi,Jakarta
1,B63077,A16227,5,Andrianto,Ekonomi,Jakarta
2,B14364,A16227,5,Andrianto,Ekonomi,Jakarta
3,B61521,A16227,4,Andrianto,Ekonomi,Jakarta
4,B62409,A16227,5,Andrianto,Ekonomi,Jakarta
...,...,...,...,...,...,...
12621,B29822,A17248,3,Junaeri,Matematika,Yogyakarta
12622,B64162,A17248,3,Junaeri,Matematika,Yogyakarta
12623,B32844,A17248,4,Junaeri,Matematika,Yogyakarta
12624,B97893,A17248,5,Junaeri,Matematika,Yogyakarta


In [ ]:
tutor_rating.Rating.value_counts()

3    4192
4    4178
5    4117
2      82
1      57
Name: Rating, dtype: int64

In [ ]:
rating_count = (tutor_rating.groupby(by = ["TutorName"])["Rating"]).count().reset_index().rename(columns = {'Rating' : 'RatingCount_Tutor'})[['TutorName', 'RatingCount_Tutor']]

In [ ]:
user_rating = pd.merge(rating_count, tutor_rating, left_on="TutorName", right_on="TutorName", how='left')
user_rating

,TutorName,RatingCount_Tutor,MenteeID,TutorID,Rating,Subject,Location
0,A. Solikhin,17,B55212,A49119,4,Fisika,Denpasar
1,A. Solikhin,17,B74603,A49119,3,Fisika,Denpasar
2,A. Solikhin,17,B76968,A49119,3,Fisika,Denpasar
3,A. Solikhin,17,B44857,A49119,4,Fisika,Denpasar
4,A. Solikhin,17,B55497,A49119,4,Fisika,Denpasar
...,...,...,...,...,...,...,...
12621,Zukri Rahman,21,B55161,A57575,4,Biologi,Jakarta
12622,Zukri Rahman,21,B86900,A57575,3,Biologi,Jakarta
12623,Zukri Rahman,21,B80176,A57575,3,Biologi,Jakarta
12624,Zukri Rahman,21,B25637,A57575,4,Biologi,Jakarta


In [ ]:
user_count = (user_rating.
     groupby(by = ['MenteeID'])['Rating'].
     count().
     reset_index().
     rename(columns = {'Rating': 'RatingCount_user'})
     [['MenteeID', 'RatingCount_user']]
    )
user_count

,MenteeID,RatingCount_user
0,B11181,6
1,B11233,5
2,B11260,6
3,B11380,11
4,B11407,2
...,...,...
1490,B99514,11
1491,B99594,8
1492,B99678,12
1493,B99800,17


In [ ]:
final_df = user_rating.merge(user_count, left_on = 'MenteeID', right_on = 'MenteeID', how = 'inner')
final_df

,TutorName,RatingCount_Tutor,MenteeID,TutorID,Rating,Subject,Location,RatingCount_user
0,A. Solikhin,17,B55212,A49119,4,Fisika,Denpasar,2
1,Zenika Pradea,20,B55212,A92767,4,Sejarah,Denpasar,2
2,A. Solikhin,17,B74603,A49119,3,Fisika,Denpasar,6
3,Firdaus,28,B74603,A15683,4,Geografi,Denpasar,6
4,Hamzah Maulana,13,B74603,A40274,3,Bahasa Indonesia,Denpasar,6
...,...,...,...,...,...,...,...,...
12621,Wagimun,27,B23406,A47594,4,Geografi,Malang,2
12622,Teguh Wahyudi,14,B92655,A80745,4,Ekonomi,Depok,1
12623,Tri Adi Eko Lestanto,21,B33630,A40985,3,Bahasa Indonesia,Denpasar,1
12624,Vika Apriliana N. A,13,B25365,A22460,4,Geografi,Yogyakarta,2


In [ ]:
avg_rating_count = rating_count.groupby("TutorName").mean()["RatingCount_Tutor"].reset_index()
avg_rating_count.rename(columns={"RatingCount_Tutor" : "avg_ratings"}, inplace=True)
avg_rating_count

,TutorName,avg_ratings
0,A. Solikhin,17.0
1,Aan Nuraini,31.0
2,Abas Suyatno,27.0
3,Abdul Kholik,21.0
4,Abdul Malik,16.0
...,...,...
490,Zenika Pradea,20.0
491,Zidane Dhevan Pratama,27.0
492,Zubaidah,26.0
493,Zukri Adinalta,19.0


In [ ]:
popular_tutor_rating = final_df.merge(avg_rating_count, left_on = "TutorName", right_on="TutorName", how="inner")
popular_tutor_rating

,TutorName,RatingCount_Tutor,MenteeID,TutorID,Rating,Subject,Location,RatingCount_user,avg_ratings
0,A. Solikhin,17,B55212,A49119,4,Fisika,Denpasar,2,17.0
1,A. Solikhin,17,B74603,A49119,3,Fisika,Denpasar,6,17.0
2,A. Solikhin,17,B76968,A49119,3,Fisika,Denpasar,8,17.0
3,A. Solikhin,17,B44857,A49119,4,Fisika,Denpasar,5,17.0
4,A. Solikhin,17,B55497,A49119,4,Fisika,Denpasar,6,17.0
...,...,...,...,...,...,...,...,...,...
12621,Oki Aditia Saputra,9,B45288,A70025,5,Bahasa Indonesia,Tangerang,5,9.0
12622,Oki Aditia Saputra,9,B34500,A70025,5,Bahasa Indonesia,Tangerang,7,9.0
12623,Oki Aditia Saputra,9,B64648,A70025,3,Bahasa Indonesia,Tangerang,8,9.0
12624,Oki Aditia Saputra,9,B64827,A70025,4,Bahasa Indonesia,Tangerang,8,9.0


In [ ]:
popular_tutor_rating.sort_values('avg_ratings', ascending=True)

,TutorName,RatingCount_Tutor,MenteeID,TutorID,Rating,Subject,Location,RatingCount_user,avg_ratings
12560,Mardiyyat Fadliellah,5,B65653,A85099,3,Sosiologi,Yogyakarta,4,5.0
12564,Mardiyyat Fadliellah,5,B11181,A85099,5,Sosiologi,Yogyakarta,6,5.0
10383,Rahmat,5,B14097,A39898,3,Kimia,Depok,9,5.0
10382,Rahmat,5,B47470,A39898,4,Kimia,Depok,8,5.0
10381,Rahmat,5,B80524,A39898,5,Kimia,Depok,7,5.0
...,...,...,...,...,...,...,...,...,...
9350,Slamet,97,B98351,A42217,3,Ekonomi,Bekasi,12,97.0
9349,Slamet,97,B64922,A65448,4,Sosiologi,Bekasi,11,97.0
9348,Slamet,97,B59661,A42217,4,Ekonomi,Bekasi,9,97.0
9354,Slamet,97,B52826,A42217,3,Ekonomi,Bekasi,10,97.0


In [ ]:
popular_tutor_rating = popular_tutor_rating[['TutorName', 'MenteeID', 'TutorID', 'Subject', 'Location', 'Rating', 'RatingCount_Tutor']]
popular_tutor_rating

,TutorName,MenteeID,TutorID,Subject,Location,Rating,RatingCount_Tutor
0,A. Solikhin,B55212,A49119,Fisika,Denpasar,4,17
1,A. Solikhin,B74603,A49119,Fisika,Denpasar,3,17
2,A. Solikhin,B76968,A49119,Fisika,Denpasar,3,17
3,A. Solikhin,B44857,A49119,Fisika,Denpasar,4,17
4,A. Solikhin,B55497,A49119,Fisika,Denpasar,4,17
...,...,...,...,...,...,...,...
12621,Oki Aditia Saputra,B45288,A70025,Bahasa Indonesia,Tangerang,5,9
12622,Oki Aditia Saputra,B34500,A70025,Bahasa Indonesia,Tangerang,5,9
12623,Oki Aditia Saputra,B64648,A70025,Bahasa Indonesia,Tangerang,3,9
12624,Oki Aditia Saputra,B64827,A70025,Bahasa Indonesia,Tangerang,4,9


#Encode Dataset

In [ ]:
from sklearn.preprocessing import LabelEncoder
# Encode user_id and mentor_id using LabelEncoder
mentee_encoder = LabelEncoder()
mentor_encoder = LabelEncoder()
popular_tutor_rating['MenteeID'] = mentee_encoder.fit_transform(popular_tutor_rating['MenteeID'])
popular_tutor_rating['TutorID'] = mentor_encoder.fit_transform(popular_tutor_rating['TutorID'])

# # Define StringLookup encoders for user_id and mentor_id
# user_id_encoder = StringLookup()
# mentor_id_encoder = StringLookup()

# # Fit encoders on the merged dataset
# user_id_encoder.adapt(popular_tutor_rating['MenteeID'])
# mentor_id_encoder.adapt(popular_tutor_rating['TutorID'])

# # Transform string user IDs and mentor IDs to numerical values
# popular_tutor_rating['MenteeID'] = user_id_encoder(popular_tutor_rating['MenteeID'])
# popular_tutor_rating['TutorID'] = mentor_id_encoder(popular_tutor_rating['TutorID'])

<ipython-input-478-c9b25467c8f6>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  popular_tutor_rating['MenteeID'] = mentee_encoder.fit_transform(popular_tutor_rating['MenteeID'])
<ipython-input-478-c9b25467c8f6>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  popular_tutor_rating['TutorID'] = mentor_encoder.fit_transform(popular_tutor_rating['TutorID'])


#Split Dataset

In [ ]:
popular_tutor_rating_df = popular_tutor_rating[['MenteeID', 'TutorID', 'Rating']]

In [ ]:
train_data, test_data = train_test_split(popular_tutor_rating_df, test_size=0.2)

#Build ML Model Using TensorFlow

In [ ]:
# Define the number of mentees, mentors
num_mentees = popular_tutor_rating_df['MenteeID'].nunique()
num_mentors = popular_tutor_rating_df['TutorID'].nunique()
# Define embedding dimensions
embedding_dim = 10

In [ ]:
# Create the input datasets
train_dataset = tf.data.Dataset.from_tensor_slices((train_data['MenteeID'], train_data['TutorID'], train_data['Rating']))
test_dataset = tf.data.Dataset.from_tensor_slices((test_data['MenteeID'], test_data['TutorID'], test_data['Rating']))

# Shuffle and batch the datasets
batch_size = 32
train_dataset = train_dataset.shuffle(len(train_data)).batch(batch_size)
test_dataset = test_dataset.batch(batch_size)

In [ ]:
# Create input layers
mentee_input = Input(shape=(1,))
mentor_input = Input(shape=(1,))

In [ ]:
# Create embedding layers
mentee_embedding = Embedding(input_dim=num_mentees, output_dim=embedding_dim)(mentee_input)
mentor_embedding = Embedding(input_dim=num_mentors, output_dim=embedding_dim)(mentor_input)

In [ ]:
# Flatten the embeddings
mentee_flattened = Flatten()(mentee_embedding)
mentor_flattened = Flatten()(mentor_embedding)

In [ ]:
# Concatenate user and mentor embeddings
concatenated = Concatenate()([mentee_flattened, mentor_flattened])

In [ ]:
# Fully connected layers
fc1 = Dense(64, activation='relu')(concatenated)
fc2 = Dense(32, activation='relu')(fc1)
output = Dense(1)(fc2)

In [ ]:
# Create the model
model = Model(inputs=[mentee_input, mentor_input], outputs=output)
model.compile(optimizer=Adam(0.001), loss='mean_squared_error', metrics=[RootMeanSquaredError()])

In [ ]:
# Train the model
model.fit(x=[train_data['MenteeID'], train_data['TutorID']], y=train_data['Rating'], epochs=10)
# Evaluate the model
test_loss = model.evaluate(x=[test_data['MenteeID'], test_data['TutorID']], y=test_data['Rating'])
print(f'Test loss: {test_loss}')

Epoch 1/10
316/316 [==============================] - 2s 3ms/step - loss: 3.1990 - root_mean_squared_error: 1.7886
Epoch 2/10
316/316 [==============================] - 1s 3ms/step - loss: 0.7031 - root_mean_squared_error: 0.8385
Epoch 3/10
316/316 [==============================] - 1s 3ms/step - loss: 0.6724 - root_mean_squared_error: 0.8200
Epoch 4/10
316/316 [==============================] - 1s 3ms/step - loss: 0.6528 - root_mean_squared_error: 0.8080
Epoch 5/10
316/316 [==============================] - 1s 3ms/step - loss: 0.6425 - root_mean_squared_error: 0.8015
Epoch 6/10
316/316 [==============================] - 1s 4ms/step - loss: 0.6349 - root_mean_squared_error: 0.7968
Epoch 7/10
316/316 [==============================] - 2s 7ms/step - loss: 0.6209 - root_mean_squared_error: 0.7880
Epoch 8/10
316/316 [==============================] - 2s 5ms/step - loss: 0.6018 - root_mean_squared_error: 0.7757
Epoch 9/10
316/316 [==============================] - 1s 4ms/step - loss: 0.5589

#Make Recommendations

In [ ]:
# Make recommendations for a specific mentee
mentee_id = 'B55212'
mentee_encoded = mentee_encoder.transform([mentee_id])[0]


# # Get mentors that the mentee has not rated
# mentors_rated = rating_df[rating_df['MenteeID'] == mentee_encoded]['TutorID'].values
# mentors = mentor_df['TutorID'].values
# mentors_not_rated = [mentor for mentor in mentors if mentor not in mentors_rated]

# # Convert mentors to their encoded indices
# mentors_encoded = [mentor_df[mentor_df['TutorID'] == mentor_id].index[0] for mentor_id in mentors_not_rated]

# # Predict ratings for mentors
# predicted_ratings = model.predict([np.full(len(mentors_encoded), mentee_encoded), np.array(mentors_encoded)])

# # Sort the mentors based on predicted ratings
# recommended_mentors = [mentor for _, mentor in sorted(zip(predicted_ratings, mentors_not_rated), reverse=True)]

# print(f"Recommended mentors for Mentee {mentee_id}:")
# for mentor in recommended_mentors:
#     print(mentor)
  
mentors = mentor_df['TutorID'].unique()
mentee_ratings = popular_tutor_rating.loc[popular_tutor_rating['TutorID'] == mentee_encoded]
mentors_rated = mentee_ratings['TutorID'].unique()
mentors_not_rated = [mentor for mentor in mentors if mentor not in mentors_rated]

#==============
mentee_input = tf.constant([mentee_encoded] * len(mentors_not_rated))
# mentor_input = tf.constant(mentor_encoder(mentors_not_rated).numpy())
mentor_input = tf.constant(mentor_encoder.transform(mentors_not_rated))
# mentor_input = []
# filtered_mentors_not_rated = []

# for mentor_id in mentors_not_rated:
#     if mentor_id in mentor_encoder.classes_:
#         mentor_input.append(mentor_encoder.transform([mentor_id])[0])
#         filtered_mentors_not_rated.append(mentor_id)

# if len(filtered_mentors_not_rated) > 0:
#     mentor_input = tf.constant([mentor_input] * len(mentee_input))
#     recommendations = model.predict([mentee_input, mentor_input]).flatten()
#     recommended_mentors = pd.DataFrame({'TutorID': filtered_mentors_not_rated, 'recommendation_score': recommendations})
#     recommended_mentors = recommended_mentors.sort_values(by='recommendation_score', ascending=False)
# else:
#     recommended_mentors = pd.DataFrame(columns=['TutorID', 'recommendation_score'])

recommendations = model.predict([mentee_input, mentor_input]).flatten()
recommended_mentors = pd.DataFrame({'TutorID': mentors_not_rated, 'recommendation_score': recommendations})
recommended_mentors = recommended_mentors.sort_values(by='recommendation_score', ascending=False)

print(recommended_mentors.head(10))

<ipython-input-491-c90975271ad0>:27: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mentors_not_rated = [mentor for mentor in mentors if mentor not in mentors_rated]


ValueError: ignored